In [2]:
from langgraph.graph import START, StateGraph, END
from typing import Annotated, TypedDict
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver

In [4]:
from google.colab import userdata
api = userdata.get('OPEN_AI')

In [5]:
llm = ChatOpenAI(api_key=api)

In [6]:
# Define State
class JokeState(TypedDict):
  topic: str
  joke: str
  explanation: str

In [7]:
# Define Nodes
def generate_joke(state: JokeState):
  prompt = f"Generate a concise joke on the following topic: \n\n {state['topic']}"
  response = llm.invoke(prompt)
  return {'joke': response.content}


def generate_explanation(state: JokeState):
  prompt = f"Generate a concise explanation of the following joke: {state['joke']}"
  response = llm.invoke(prompt)
  return {'explanation': response.content}

In [13]:
# Implement Persistence
checkpointer = InMemorySaver()

In [14]:
# Define Workflow
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

workflow = graph.compile(checkpointer=checkpointer)

In [15]:
# We will use threadID here which acts like a primary key to store conversations in database
initial_state = {'topic': 'Seattle'}

In [18]:
# These enable different coversations
config1 = {'configurable': {'thread_id': '1'}}
config2 = {'configurable': {'thread_id': '2'}}

In [19]:
# Executing workflow and tagging it with thread id
workflow.invoke({'topic': 'Seattle'}, config=config1)

{'topic': 'Seattle',
 'joke': 'Why do Seattleites always carry an umbrella? \n\nIn case it drizzles!',
 'explanation': "This joke plays on the reputation of Seattle for frequent rain and drizzle. The punchline suggests that Seattleites always carry an umbrella in case there is even a slight drizzle, highlighting the stereotype of their affinity for umbrellas due to the city's notoriously rainy weather."}

In [20]:
workflow.get_state(config=config1)

StateSnapshot(values={'topic': 'Seattle', 'joke': 'Why do Seattleites always carry an umbrella? \n\nIn case it drizzles!', 'explanation': "This joke plays on the reputation of Seattle for frequent rain and drizzle. The punchline suggests that Seattleites always carry an umbrella in case there is even a slight drizzle, highlighting the stereotype of their affinity for umbrellas due to the city's notoriously rainy weather."}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f072ee1-0b90-6b98-8002-dc843f61d99f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-06T17:52:05.683058+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f072ee1-0452-6ac3-8001-0bb9bf54f94c'}}, tasks=(), interrupts=())

In [22]:
list(workflow.get_state_history(config=config1))

[StateSnapshot(values={'topic': 'Seattle', 'joke': 'Why do Seattleites always carry an umbrella? \n\nIn case it drizzles!', 'explanation': "This joke plays on the reputation of Seattle for frequent rain and drizzle. The punchline suggests that Seattleites always carry an umbrella in case there is even a slight drizzle, highlighting the stereotype of their affinity for umbrellas due to the city's notoriously rainy weather."}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f072ee1-0b90-6b98-8002-dc843f61d99f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-06T17:52:05.683058+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f072ee1-0452-6ac3-8001-0bb9bf54f94c'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'Seattle', 'joke': 'Why do Seattleites always carry an umbrella? \n\nIn case it drizzles!'}, next=('generate_explanation',), config={'configurable': 

In [23]:
# Starting conversation using a different thread id


workflow.invoke({'topic': 'Paris'}, config=config2)

{'topic': 'Paris',
 'joke': "Why did the French chef only use fresh ingredients in Paris? \n\nBecause he couldn't find thyme to kill!",
 'explanation': 'This joke is a play on words, combining the French word for time, "temps," with the English word for the herb thyme. The chef in Paris only used fresh ingredients because he couldn\'t find the time (thyme) to kill.'}

In [25]:
list(workflow.get_state_history(config=config2))

[StateSnapshot(values={'topic': 'Paris', 'joke': "Why did the French chef only use fresh ingredients in Paris? \n\nBecause he couldn't find thyme to kill!", 'explanation': 'This joke is a play on words, combining the French word for time, "temps," with the English word for the herb thyme. The chef in Paris only used fresh ingredients because he couldn\'t find the time (thyme) to kill.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f072ee6-f28a-64c9-8002-16b94a716104'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-06T17:54:44.120257+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f072ee6-e8ea-67ab-8001-2481f16a1738'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'Paris', 'joke': "Why did the French chef only use fresh ingredients in Paris? \n\nBecause he couldn't find thyme to kill!"}, next=('generate_explanation',), config={'configurable': {'thr

## **Time Travel**

In [27]:
# I want to go back to the state where I had the topic but no joke generated
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'Paris', 'joke': "Why did the French chef only use fresh ingredients in Paris? \n\nBecause he couldn't find thyme to kill!", 'explanation': 'This joke is a play on words, combining the French word for time, "temps," with the English word for the herb thyme. The chef in Paris only used fresh ingredients because he couldn\'t find the time (thyme) to kill.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f072ee6-f28a-64c9-8002-16b94a716104'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-06T17:54:44.120257+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f072ee6-e8ea-67ab-8001-2481f16a1738'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'Paris', 'joke': "Why did the French chef only use fresh ingredients in Paris? \n\nBecause he couldn't find thyme to kill!"}, next=('generate_explanation',), config={'configurable': {'thr

In [29]:
list(workflow.get_state_history(config={'configurable': {'thread_id': '2', 'checkpoint_id':'1f072ee6-de19-6957-8000-6d7b97fc11de'}}))

[StateSnapshot(values={'topic': 'Paris'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f072ee6-de19-6957-8000-6d7b97fc11de'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-08-06T17:54:41.976947+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f072ee6-de16-6054-bfff-40267f49ffcc'}}, tasks=(PregelTask(id='9c454bbf-12d1-6c9e-c2fb-cd0358ccdb16', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': "Why did the French chef only use fresh ingredients in Paris? \n\nBecause he couldn't find thyme to kill!"}),), interrupts=())]

In [30]:
# Let's start the execution from this checkpoint again
workflow.invoke(None, config={'configurable': {'thread_id': '2', 'checkpoint_id':'1f072ee6-de19-6957-8000-6d7b97fc11de'}})

{'topic': 'Paris',
 'joke': "Why did the Eiffel Tower break up with Paris? Because it couldn't handle the long-distance relationship!",
 'explanation': 'The joke plays on the idea of the Eiffel Tower being in a relationship with the city of Paris due to their strong association. The punchline humorously suggests that the Eiffel Tower "broke up" with Paris because it couldn\'t physically handle the distance as it is a prominent landmark in the city.'}

In [33]:
# We get two new states appended
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'Paris', 'joke': "Why did the Eiffel Tower break up with Paris? Because it couldn't handle the long-distance relationship!", 'explanation': 'The joke plays on the idea of the Eiffel Tower being in a relationship with the city of Paris due to their strong association. The punchline humorously suggests that the Eiffel Tower "broke up" with Paris because it couldn\'t physically handle the distance as it is a prominent landmark in the city.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f072f5f-affa-62e1-8002-cea15d4a37c6'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-06T18:48:45.209648+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f072f5f-92e9-65b5-8001-008c6f33b2a2'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'Paris', 'joke': "Why did the Eiffel Tower break up with Paris? Because it couldn't handle the long-di

In [45]:
workflow.update_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f072ee6-de19-6957-8000-6d7b97fc11de", "checkpoint_ns": ""}}, {'topic':'Bubble tea'})

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f072f81-584c-6416-8001-6f091248a0a4'}}

In [46]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'Bubble tea'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f072f81-584c-6416-8001-6f091248a0a4'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-08-06T19:03:48.696353+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f072ee6-de19-6957-8000-6d7b97fc11de'}}, tasks=(PregelTask(id='49f09da1-0c11-adef-5b2a-caf154c5cd1f', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f072f7f-4c89-6c77-8001-25e7862d81e0'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-08-06T19:02:53.776173+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoin

In [47]:
workflow.invoke(None, config = {'configurable':{'thread_id':'2', 'checkpoint_id':'1f072f81-584c-6416-8001-6f091248a0a4'}})

{'topic': 'Bubble tea',
 'joke': 'Why did the bubble tea blush? Because it saw the tapioca pearls at the bottom!',
 'explanation': 'This joke plays on the idea of the bubble tea blushing (turning red) when it sees the tapioca pearls at the bottom, as if it is embarrassed or shy about them. It is a light-hearted and punny explanation for why the bubble tea appears to have changed color.'}

In [48]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'Bubble tea', 'joke': 'Why did the bubble tea blush? Because it saw the tapioca pearls at the bottom!', 'explanation': 'This joke plays on the idea of the bubble tea blushing (turning red) when it sees the tapioca pearls at the bottom, as if it is embarrassed or shy about them. It is a light-hearted and punny explanation for why the bubble tea appears to have changed color.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f072f88-dd01-6631-8003-6b3cd6e81fdb'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-08-06T19:07:10.516561+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f072f88-d336-6b80-8002-73fea46fa43d'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'Bubble tea', 'joke': 'Why did the bubble tea blush? Because it saw the tapioca pearls at the bottom!'}, next=('generate_explanation',), config={'configurable': {'thr